A neural network consist of cnn layer (Kim,2014) and 4 fully connected layers.

Source: https://github.com/jojonki/cnn-for-sentence-classification





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/sharif/DeepLearning/ipython(guide)')

In [ ]:
import numpy as np
import codecs
import os
import random
import pandas
from keras import backend as K
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Lambda, Permute, Dropout,Softmax
from keras.layers import LSTM, MaxPooling1D
from keras.optimizers import SGD
import ast
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import gensim
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
limit_number = 750
data = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv',index_col=0,converters={'body': eval})
data = data.dropna().reset_index(drop=True)
X = data["body"].values.tolist()
y = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv')
labels = []
tag=[]
for item in y['tag']:
  labels += [i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' ']
  tag.append([i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' '])
labels = list(set(labels))
mlb = MultiLabelBinarizer()
Y=mlb.fit_transform(tag)

In [ ]:
len(labels)

78

In [ ]:
sentence_maxlen = max(map(len, (d for d in X)))
print('sentence maxlen', sentence_maxlen)

sentence maxlen 300


In [ ]:
freq_dist = pandas.read_csv('../Data/FreqDist_sorted.csv',index_col=False)
vocab=[]
for item in freq_dist["word"]:
  try:
    word=re.sub(r"[\u200c-\u200f]","",item.replace(" ",""))
    vocab.append(word)
  except:
    pass
  
print(vocab[10])

زبان


In [ ]:
vocab = sorted(vocab)
vocab_size = len(vocab)

In [ ]:
print('vocab size', len(vocab))
w2i = {w:i for i,w in enumerate(vocab)}
# i2w = {i:w for i,w in enumerate(vocab)}
print(w2i["زبان"])

vocab size 225345
129280


In [ ]:
def vectorize(data, sentence_maxlen, w2i):
    vec_data = []
    
    for d in data:
       
        vec = [w2i[w] for w in d if w in w2i]
        pad_len = max(0, sentence_maxlen - len(vec))
        vec += [0] * pad_len
        vec_data.append(vec)
        # print(d)
        
    vec_data = np.array(vec_data)
    
    return vec_data

vecX = vectorize(X, sentence_maxlen, w2i)
vecY=Y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vecX, vecY, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
print('train: ', X_train.shape , '\ntest: ', X_test.shape , '\nval: ', X_val.shape ,"\ny_tain:",y_train.shape )
# print(vecX[0])

train:  (12935, 300) 
test:  (4312, 300) 
val:  (4312, 300) 
y_tain: (12935, 78)


In [ ]:
embd_dim = 300


# ***If the word2vec model is not generated before, we should run the next block.***

In [ ]:
# embed_model = gensim.models.Word2Vec(X, size=embd_dim, window=5, min_count=5)
# embed_model.save('word2vec_model')

# ***Otherwise, we can run the next block.***

In [ ]:
embed_model=gensim.models.Word2Vec.load('word2vec_model')

In [ ]:
word2vec_embd_w = np.zeros((vocab_size, embd_dim))
for word, i in w2i.items():
  if word in embed_model.wv.vocab:
        embedding_vector =embed_model[word]
  
        # words not found in embedding index will be all-zeros.
        word2vec_embd_w[i] = embedding_vector

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [ ]:
def Net(vocab_size, embd_size, sentence_maxlen, glove_embd_w):
    sentence = Input((sentence_maxlen,), name='SentenceInput')
    
    # embedding
    embd_layer = Embedding(input_dim=vocab_size, 
                           output_dim=embd_size, 
                           weights=[word2vec_embd_w], 
                           trainable=False,
                           name='shared_embd')
    embd_sentence = embd_layer(sentence)
    # embd_sentence = Permute((2,1))(embd_sentence)
    # embd_sentence = Lambda(lambda x: K.expand_dims(x, -1))(embd_sentence)
    
    # lstm
    lstm_1 = LSTM(128, return_sequences=True,name='lstm_1')(embd_sentence)
    # drop_out_1 = Dropout(0.5,name='drop_out_1')(lstm_1)
    lstm_2 = LSTM(64,name='lstm_2')(lstm_1)
    drop_out_2 = Dropout(0.4,name='drop_out_2')(lstm_2)

    # dense
    dense_1 = Dense(256,name='dense_1')(drop_out_2)
    dense_2 = Dense(128,name='dense_2')(dense_1)
    dense_3 = Dense(len(labels),activation='sigmoid',name='dense_3')(dense_1)
    out = Softmax(name='out')(dense_3) 

    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model = Model(inputs=sentence, outputs=out, name='sentence_claccification')
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=["accuracy", "binary_accuracy",
    "categorical_accuracy",]) 
    return model


model = Net(vocab_size, embd_dim, sentence_maxlen,word2vec_embd_w)
print(model.summary())

Model: "sentence_claccification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SentenceInput (InputLayer)   [(None, 300)]             0         
_________________________________________________________________
shared_embd (Embedding)      (None, 300, 300)          67603500  
_________________________________________________________________
lstm_1 (LSTM)                (None, 300, 128)          219648    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
drop_out_2 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               16640     
_________________________________________________________________
dense_3 (Dense)              (None, 78)    

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5) # Model stop training after 50 epoch where validation loss didnt decrease
mc = ModelCheckpoint('best_cnn_4fc.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True) #You save model weight at the epoch where validation loss is minimal
model.fit(X_train, y_train, batch_size=32,epochs=50,verbose=1,validation_data=(X_val, y_val),callbacks=[es,mc])#you can run for 1000 epoch btw model will stop after 50 epoch without better validation loss

Epoch 1/50
405/405 [==============================] - 259s 631ms/step - loss: 7.8861 - accuracy: 0.0198 - binary_accuracy: 0.9767 - categorical_accuracy: 0.0198 - val_loss: 7.7035 - val_accuracy: 0.0079 - val_binary_accuracy: 0.9769 - val_categorical_accuracy: 0.0079

Epoch 00001: val_loss improved from inf to 7.70349, saving model to best_cnn_4fc.h5
Epoch 2/50
405/405 [==============================] - 261s 645ms/step - loss: 7.8361 - accuracy: 0.0114 - binary_accuracy: 0.9765 - categorical_accuracy: 0.0114 - val_loss: 7.7148 - val_accuracy: 0.0079 - val_binary_accuracy: 0.9769 - val_categorical_accuracy: 0.0079

Epoch 00002: val_loss did not improve from 7.70349
Epoch 3/50
405/405 [==============================] - 260s 643ms/step - loss: 7.7854 - accuracy: 0.0136 - binary_accuracy: 0.9767 - categorical_accuracy: 0.0136 - val_loss: 7.7143 - val_accuracy: 0.0077 - val_binary_accuracy: 0.9769 - val_categorical_accuracy: 0.0077

Epoch 00003: val_loss did not improve from 7.70349
Epoch 4

# ***If the model is generated before:***

In [ ]:
# model = load_model('Lstm_with_binary.h5')
model.save('Lstm_with_binary.h5')

In [ ]:
pred=model.predict(X_test)
# For evaluation: If the probability > 0.5 you can say that it belong to the class.

In [ ]:
y_test[0].argmax() , pred[0][63]

(63, 0.014853852)

In [ ]:
print(pred[0])#example

[0.01485386 0.01485373 0.01485382 0.00559326 0.01051748 0.00554261
 0.01485351 0.01195763 0.00687896 0.01485349 0.01485391 0.00547362
 0.00546706 0.00557473 0.00560065 0.014854   0.014854   0.01485329
 0.01485336 0.01482902 0.01485367 0.01485364 0.00791699 0.014854
 0.01485037 0.01485378 0.01485408 0.01485378 0.01485364 0.01485387
 0.01401732 0.01485049 0.014854   0.01485401 0.01485179 0.01485395
 0.01485368 0.01272168 0.01176419 0.00616108 0.01485365 0.01485358
 0.01485171 0.01485401 0.01485336 0.00577146 0.00724477 0.01485395
 0.01485383 0.01484488 0.00659727 0.00567235 0.01138096 0.01485317
 0.01485349 0.01357225 0.01433295 0.01177621 0.01485317 0.01485409
 0.01485357 0.01446518 0.01485381 0.01485385 0.01485257 0.01485388
 0.00746382 0.01485345 0.01485356 0.01485104 0.01485414 0.01485365
 0.01485381 0.00837003 0.00582638 0.01485405 0.01484641 0.01485371]


In [ ]:
y_pred=[]
measure = np.mean(pred[0]) + .5*np.sqrt(np.var(pred[0]))
for l in pred:
  temp=[]
  for value in l:
    if value>=measure:
      temp.append(1)
    else:
      temp.append(0)
  y_pred.append(temp)

In [ ]:
from sklearn.metrics import classification_report,accuracy_score

print("accuracy=",accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

accuracy= 0.0
              precision    recall  f1-score   support

           0       0.03      0.70      0.05       119
           1       0.05      0.89      0.09       164
           2       0.04      0.84      0.08       158
           3       0.00      0.00      0.00        16
           4       0.50      0.01      0.02        84
           5       0.00      0.00      0.00         9
           6       0.04      0.87      0.08       159
           7       0.00      0.00      0.00        42
           8       0.01      0.13      0.01        23
           9       0.03      0.85      0.05       107
          10       0.03      0.70      0.06       151
          11       0.12      0.81      0.21        70
          12       0.27      1.00      0.42       130
          13       0.00      0.00      0.00        11
          14       0.00      0.10      0.01        10
          15       0.03      0.62      0.06       153
          16       0.02      0.51      0.04       130
          17 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
